In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../../')
print(os.getcwd())
from v2.strategy.indicators.optimal_v2 import Optimal_v2
from v2.model import Trading
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.notebook_utils import fetchIndicators, genDataForAll
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier



/mnt/c/Users/jam60/OneDrive/Repos/vivaldi/back_testing


In [7]:
def load_config():
    my_config = {}
    with open('config.config') as config:
        for line in config:
            args = line.split('=')
            my_config[args[0]] = args[1].rstrip().split(',')
    return my_config

In [8]:
model = Trading(load_config())
my_inds = fetchIndicators(["rsi", "macd", "stochastic_oscillator", "optimal_v2", 'slope', 'variance', 'bollinger_bands', 'smma'])
dataset_list = []
for d in model.dfs:
    cur_dataset = d[0]
    genDataForAll(cur_dataset, my_inds)
    cur_dataset.dropna(inplace=True)
    mm_scalar = MinMaxScaler()
    cur_dataset[["rsi", "stosc_k", "stosc_d"]] = mm_scalar.fit_transform(cur_dataset[["rsi", "stosc_k", "stosc_d"]])
    dataset_list.append(cur_dataset)
final_dataset = pd.concat(dataset_list)

In [11]:
import pickle
from tensorflow.keras.models import load_model
sell_model = pickle.load(open('./v2/strategy/saved_models/optimal_v2_sell_rf_90.sav', 'rb'))
buy_rf = pickle.load(open('./v2/strategy/saved_models/optimal_classify/troughs_random_forest.sav', 'rb'))
buy_boost = pickle.load(open('./v2/strategy/saved_models/optimal_classify/troughs_boost.sav', 'rb'))
buy_lstm = load_model('./v2/strategy/saved_models/optimal_classify/troughs_lstm.sav')

In [15]:
rf_data = final_dataset[['stosc_k', 'slope', 'macd_diff', 'rsi']]
final_dataset['rf_predict'] = buy_rf.predict_proba(rf_data)[:,1]
boost_data = final_dataset[['stosc_k', 'slope', 'macd_diff', 'rsi', 'rf_predict']]
final_dataset['boost_predict'] = buy_boost.predict_proba(boost_data)[:,1]
lstm_data = final_dataset[['stosc_k', 'slope', 'macd_diff', 'rsi', 'rf_predict', 'boost_predict']].values
lstm_data = np.reshape(lstm_data, (lstm_data.shape[0], 1, lstm_data.shape[1]))
final_dataset['lstm_predict'] = buy_lstm.predict_proba(lstm_data)[:,1]


Instructions for updating:
Please use `model.predict()` instead.


In [16]:
sell_model_data = final_dataset[["close", "rsi", "MACD", "stosc_k", "stosc_d", "slope", "variance"]]

In [17]:
final_dataset['sell_predict'] = sell_model.predict(sell_model_data.values)


In [18]:
final_dataset

,time,open,high,low,close,volume,trades,rsi_diff,rsi_u,rsi_d,...,variance,boll_sma,boll_stdev,boll_upper,boll_lower,SMMA,rf_predict,boost_predict,lstm_predict,sell_predict
3498264,1590991140,9527.9,9527.9,9524.3,9529.982374,10.857477,13,-2.7,0.0,2.7,...,51.135794,9545.242216,7.150930,9559.544075,9530.940357,9542.632866,0.8,0.988336,0.999992,0.0
3498265,1590991200,9527.1,9533.5,9527.1,9529.841187,8.735651,19,2.6,2.6,0.0,...,50.727922,9544.891108,7.122354,9559.135815,9530.646401,9542.488144,0.8,0.988336,0.999991,0.0
3498266,1590991260,9529.7,9529.7,9526.0,9527.920593,4.977061,15,-3.7,0.0,3.7,...,51.086502,9544.528887,7.147482,9558.823852,9530.233923,9542.323364,0.8,0.988336,0.999988,0.0
3498267,1590991320,9526.0,9529.7,9526.0,9528.810297,4.557759,10,3.7,3.7,0.0,...,52.367533,9544.229999,7.236541,9558.703082,9529.756916,9542.170542,0.8,0.988336,0.999987,0.0
3498268,1590991380,9529.7,9535.6,9529.7,9532.205148,2.057243,15,5.9,5.9,0.0,...,53.021702,9543.996111,7.281600,9558.559311,9529.432910,9542.057864,0.8,0.988336,0.999989,0.0
3498269,1590991440,9534.1,9534.1,9533.0,9532.602574,0.384009,3,-2.6,0.0,2.6,...,53.717636,9543.780833,7.329232,9558.439296,9529.122370,9541.950975,0.8,0.988336,0.999988,0.0
3498270,1590991500,9533.1,9533.1,9533.1,9532.851287,0.090000,3,0.1,0.1,0.0,...,54.366580,9543.574861,7.373370,9558.321600,9528.828122,9541.848125,0.8,0.988336,0.999989,0.0
3498271,1590991560,9537.3,9537.3,9537.3,9535.075644,0.033000,1,4.2,4.2,0.0,...,54.384224,9543.389097,7.374566,9558.138229,9528.639965,9541.771593,0.8,0.988336,0.999990,0.0
3498272,1590991620,9537.4,9537.4,9537.4,9536.237822,0.005000,1,0.1,0.1,0.0,...,53.138366,9543.170104,7.289607,9557.749318,9528.590891,9541.709072,0.8,0.988336,0.999992,0.0
3498273,1590991680,9541.0,9541.0,9541.0,9538.618911,0.172647,1,3.6,3.6,0.0,...,51.916982,9542.994497,7.205344,9557.405185,9528.583808,9541.674165,0.8,0.988336,0.999993,0.0
